In [2]:
from PIL import Image
import numpy as np
import os
from osgeo import gdal
import cv2
import matplotlib.pyplot as plt

In [3]:
# Dict used to access bands_arr
BANDS = {
    'B1' : 0,
    'B2' : 1,
    'B3' : 2,
    'B4' : 3,
    'B5' : 4,
    'B6' : 5,
    'B7' : 6,
    'B8' : 7,
    'B8A' : 8,
    'B9' : 9,
    'B10' : 10,
    'B11' : 11,
    'B12' : 12,
}

In [4]:
import json
with open('./band_threshold.json') as f:
    BAND_THRESHOLD = json.load(f)

In [5]:
# Gets raw data as parameter and return processed image data as np array
# calculate thresholds from data histogram and interpolate the range to 0 to 255

def adjust_img(image, band):
    band = band.replace('B0', '').replace('B', '')

    mn = BAND_THRESHOLD[f'B{band}'][0]
    mx = BAND_THRESHOLD[f'B{band}'][1]

    adj_img = np.interp(image, [mn, mx], [0,255], left=0, right=255)
    # print(thres, mn, mx, image.max(), adj_img.max())

    return adj_img

In [6]:
# Opens the files in fdirlis using gdal and return the image data after adjustments
# in form of array of numpy arrays

def get_bands(fdirlis, dir = ''):
    bands_arr = []
    for band in fdirlis:
        print('\t', band)
        band_name = band.split('_')[-1][:-4]

        ds = gdal.Open(dir + band)
        rb = ds.GetRasterBand(1)
        img_array = rb.ReadAsArray()
        adjusted_img = adjust_img(img_array, band_name)
        
        # print(adjusted_img.max())
        bands_arr.append(adjusted_img)
    return bands_arr

In [7]:
# Divide the given image to num^2 images and save them as png with 
# its file number at end

def crop_img(img, num, save_prefix):
    size = img.size
    crop_sizes = []
    for i in range(num):
        for j in range(num):
            x1 = i * size[0]//num
            x2 = x1 + size[0]//num
            y1 = j * size[1]//num
            y2 = y1 + size[1]//num
            crop_sizes.append((x1, y1, x2, y2))

    for i, s in enumerate(crop_sizes):
        cropped = img.crop(s)
        cropped.save(save_prefix + f'{i}.jpg', format='JPEG', quality=80)

In [8]:
# Receive bands array and band name to resize and save cropped grayscale images into directory

def single_band_crop(bands_arr, band, save_prefix, size):
    # print("\t", band)
    img = Image.fromarray(bands_arr[BANDS[band]].astype('uint8'), mode='L').resize((size, size), resample=Image.NEAREST)
    crop_img(img, 4, save_prefix + f'_{band}_')

In [9]:
# Receive bands array and band names in order to resize and save cropped RGB images into directory

def rgb_crop(bands_arr, b1, b2, b3, save_prefix, size):
    
    red = Image.fromarray(bands_arr[BANDS[b1]].astype('uint8'), mode='L').resize((size, size), resample=Image.NEAREST)
    green = Image.fromarray(bands_arr[BANDS[b2]].astype('uint8'), mode='L').resize((size, size), resample=Image.NEAREST)
    blue = Image.fromarray(bands_arr[BANDS[b3]].astype('uint8'), mode='L').resize((size, size), resample=Image.NEAREST)
    img = Image.merge("RGB", (red, green, blue))
    
    crop_img(img, 4, save_prefix + f'_%s_%s_%s_'% (b1,b2,b3))

In [10]:
# dir_path = 'extracted/1918959_5ee6c9c4d2041265_201911_Wildfire_AUS_SN02S33_021653E151_4496882019112120191121MS00_GG001002003/S2A_MSIL1C_20191121T000241_N0208_R030_T56HLJ_20191121T011741.SAFE/GRANULE/L1C_T56HLJ_A023050_20191121T000241/IMG_DATA/'
# # dir_path = 'extracted/1918161_5ee6c9c4d2041265_201911_Wildfire_AUS_SN02S31_225762E152_5290432019111620191116MS00_GG001002003/S2B_MSIL1C_20191116T000239_N0208_R030_T56JML_20191116T011508.SAFE/GRANULE/L1C_T56JML_A014070_20191116T000238/IMG_DATA/'
# flist = os.listdir(dir_path)
# flist[:-1]

In [11]:
# b_arr = get_bands(flist[:-1], dir_path)

In [12]:
IMG_SIZE = 10980

In [13]:
# single_band_crop(b_arr, 'B10', 'output/sample', IMG_SIZE)

In [14]:
# rgb_crop(b_arr, 'B4', 'B3', 'B2', './output/samplergb', IMG_SIZE)

In [15]:
from natsort import natsorted
import threading

In [16]:
if not os.path.exists("./cropped_new"):
    os.makedirs("./cropped_new")
if not os.path.exists("./cropped_new/true_color"):
    os.makedirs("./cropped_new/true_color")
if not os.path.exists("./cropped_new/infrared/"):
    os.makedirs("./cropped_new/infrared/")
if not os.path.exists("./cropped_new/atm_penetration/"):
    os.makedirs("./cropped_new/atm_penetration/")
if not os.path.exists("./cropped_new/bands/"):
    os.makedirs("./cropped_new/bands/")

In [18]:
import glob
files = glob.glob('./cropped_new/*/*.jpg')
# files.extend(glob.glob('./cropped_new/*/bin/*.jpg'))
done_scenes = []
for f in files:
    done_scenes.append('_'.join(f.split('/')[-1].split('_')[:-2]))
for s in done_scenes:
    c = done_scenes.count(s)
    if c != 208 and c != 0:
        # print(s, c)
        done_scenes = [i for i in done_scenes if i != s]
done_scenes = set(done_scenes)

In [19]:
# iterate over all folders in extracted that contains band data
threads = []
c = 0
for path, dir, files in os.walk('/media/navroop/New Volume1/wildfire/new'):
    if len(files) == 14:
        files = natsorted(files)[:-1]
        scene_id = path.split('/')[-2]
        print(c, '  ', scene_id)
        c += 1
        # print(files)
        if scene_id in done_scenes:
            # print(scene_id)
            continue

        bands_arr = get_bands(files, path+'/')

        # save all bands cropped to given dir
        for band in BANDS:
            threads.append(threading.Thread(target=single_band_crop, args=(
                bands_arr, band, f'./cropped_new/bands/{scene_id}', IMG_SIZE)))
            # single_band_crop(bands_arr, band, f'./cropped/{scene_id}', IMG_SIZE)

        # save cropped rgb image using given bands to given dir
        threads.append(threading.Thread(target=rgb_crop, args=(
            bands_arr, 'B4', 'B3', 'B2', f'./cropped_new/true_color/{scene_id}', IMG_SIZE)))
        threads.append(threading.Thread(target=rgb_crop, args=(
            bands_arr, 'B12', 'B11', 'B8A', f'./cropped_new/atm_penetration/{scene_id}', IMG_SIZE)))
        threads.append(threading.Thread(target=rgb_crop, args=(
            bands_arr, 'B8', 'B4', 'B3', f'./cropped_new/infrared/{scene_id}', IMG_SIZE)))
        # rgb_crop(bands_arr, 'B4', 'B3', 'B2',f'./cropped/true_color/{scene_id}')
        # rgb_crop(bands_arr, 'B12', 'B11', 'B8A',f'./cropped/atm_penetration/{scene_id}')
        # rgb_crop(bands_arr, 'B8', 'B4', 'B3', f'./cropped/infrared/{scene_id}')

        for t in threads:
            t.start()
        for t in threads:
            t.join()
        threads.clear()


0    L1C_T51VWK_A031861_20210729T031543
	 T51VWK_20210729T031541_B01.jp2
	 T51VWK_20210729T031541_B02.jp2
	 T51VWK_20210729T031541_B03.jp2
	 T51VWK_20210729T031541_B04.jp2
	 T51VWK_20210729T031541_B05.jp2
	 T51VWK_20210729T031541_B06.jp2
	 T51VWK_20210729T031541_B07.jp2
	 T51VWK_20210729T031541_B08.jp2
	 T51VWK_20210729T031541_B8A.jp2
	 T51VWK_20210729T031541_B09.jp2
	 T51VWK_20210729T031541_B10.jp2
	 T51VWK_20210729T031541_B11.jp2
	 T51VWK_20210729T031541_B12.jp2
1    L1C_T51VWJ_A031818_20210726T030546
	 T51VWJ_20210726T030551_B01.jp2
	 T51VWJ_20210726T030551_B02.jp2
	 T51VWJ_20210726T030551_B03.jp2
	 T51VWJ_20210726T030551_B04.jp2
	 T51VWJ_20210726T030551_B05.jp2
	 T51VWJ_20210726T030551_B06.jp2
	 T51VWJ_20210726T030551_B07.jp2
	 T51VWJ_20210726T030551_B08.jp2
	 T51VWJ_20210726T030551_B8A.jp2
	 T51VWJ_20210726T030551_B09.jp2
	 T51VWJ_20210726T030551_B10.jp2
	 T51VWJ_20210726T030551_B11.jp2
	 T51VWJ_20210726T030551_B12.jp2
2    L1C_T52VDQ_A031918_20210802T025547
	 T52VDQ_20210802T0255